In [1]:
# Step 1 - Import Chat Model and Configure the API Key

from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/openai_key.txt')
OPENAI_API_KEY = f.read()

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=1, streaming=True)

In [2]:
# Step 2 - Chat Prompt Template

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages(
    [
        # 1. System Prompt based on your old RAG use case
        SystemMessage(
            content = ("""You are a helpful assistant for answering questions about the MS in Computer Science program 
                        at CSUDH. Answer the user's question only using the given context and past history list of messages""")
        ),
        
        # 2. Chat history placeholder
        MessagesPlaceholder(
            variable_name = "history"
        ),
        
        # 3. Human input formatted with RAG context
        HumanMessagePromptTemplate.from_template(
            "Context: {context}\n\nQuestion: {human_input}"
        ),
    ]
)


In [3]:
# Step 3 - Create a Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [4]:
# Step 4 - Initialize an embedding_model and ChromaDB Connection

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

In [5]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [6]:
from langchain_core.runnables import RunnableLambda

def get_context_from_retriever(input_dict):
    human_input = input_dict["human_input"]
    docs = retriever.invoke(human_input)
    context_str = "\n\n".join(doc.page_content for doc in docs)
    return {"context": context_str}  

context_runnable = RunnableLambda(get_context_from_retriever)

In [7]:
# Step 4 - Initialize the Memory
from langchain_core.runnables import RunnableLambda

memory_buffer = {"history": []}

def get_history_from_buffer(human_input):
    return memory_buffer["history"]

runnable_get_history_from_buffer = RunnableLambda(get_history_from_buffer)

In [8]:
from langchain_core.runnables import RunnablePassthrough

# Assign both context and chat_history
chain = RunnablePassthrough.assign(
        history=runnable_get_history_from_buffer,  # memory
        context=context_runnable                        # rag context from retriever
    ) | chat_template | chat_model | output_parser

In [9]:
# Step 8 - Run Step 6 and 7 in a loop

while True:
    
    # User Input
    query = {"human_input" : input('Enter your input: ')}
    print(f"*User: {query['human_input']}")

    # Exit
    if query["human_input"] in ['bye', 'quit', 'exit']:
        break

    # invoke chain 
    response = chain.invoke(query)
    print(f"*AI: {response}")

    # Updating memory
    memory_buffer["history"].append(HumanMessage(content=query["human_input"]))
    memory_buffer["history"].append(AIMessage(content=response))

Enter your input:  hi


*User: hi
*AI: Hello! How can I assist you today?


Enter your input:  what is your name?


*User: what is your name?
*AI: I don't have a personal name, but I'm here to help you with information about the MS in Computer Science program at CSUDH. How can I assist you today?


Enter your input:  i want to give you a name


...

*User: do you remember your creator name?
*AI: I don't have information about my creator's name. However, I'm here to assist you with your questions about the MS in Computer Science program at CSUDH. How can I help you?


Enter your input:  your creator is AKhil Padma from CSUDH CS


*User: your creator is AKhil Padma from CSUDH CS
*AI: Thank you for that information! If you have any questions or need assistance regarding the MS in Computer Science program at CSUDH, feel free to ask.


Enter your input:  bye


*User: bye


## History

In [10]:
memory_buffer['history']

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I don't have a personal name, but I'm here to help you with information about the MS in Computer Science program at CSUDH. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i want to give you a name', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Sure! Feel free to give me a name, and I'll respond to it. What name do you have in mind?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='cheeku is your name', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cheeku it is! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what are the office timings of bhrigu celly

In [11]:
import pickle

chat_history = pickle.dumps(memory_buffer)

with open("chats_data/conversation_memory.pkl", "wb") as f:
    f.write(chat_history)

In [12]:
chat_history_loaded = pickle.load(open("chats_data/conversation_memory.pkl", "rb"))

In [13]:
chat_history_loaded

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't have a personal name, but I'm here to help you with information about the MS in Computer Science program at CSUDH. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='i want to give you a name', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Sure! Feel free to give me a name, and I'll respond to it. What name do you have in mind?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='cheeku is your name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Cheeku it is! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what are the office tim